In [ ]:
!pip install ultralytics deep-sort-realtime opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 103.8 MB/s eta 0:00:00


In [ ]:
import torch, ultralytics.nn.modules
torch.serialization.add_safe_globals([ultralytics.nn.modules.Conv])


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort


In [ ]:
# Load YOLO model
model = YOLO("yolov8n.pt")

# Initialize DeepSORT tracker
tracker = DeepSort(
    max_age=30,
    n_init=2,
    max_iou_distance=0.7
)


In [ ]:
input_video = "/content/EV20250426-144029-000346F (1).mp4"
output_video = "/content/yolo+deepsort.mp4"

cap = cv2.VideoCapture(input_video)

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 30.0,
                      (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO detection
    results = model(frame, verbose=False)[0]

    dets = []
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        dets.append(([x1, y1, x2-x1, y2-y1], conf, cls))

    # DeepSORT tracking
    tracks = tracker.update_tracks(dets, frame=frame)

    # Draw tracks
    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = track.to_ltrb()
        track_id = track.track_id

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)),
                      (0,255,0), 2)
        cv2.putText(frame, f"ID {track_id}",
                    (int(x1), int(y1)-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print("✓ Tracking complete! File saved as:", output_video)


✓ Tracking complete! File saved as: /content/yolo+deepsort.mp4


In [ ]:
import json

coco = {
    "images": [],
    "annotations": [],
    "categories": [{"id": 1, "name": "object"}]
}

ann_id = 1
frames = sorted(set([r["frame"] for r in tracking_results]))

# ADD image entries
for frame in frames:
    coco["images"].append({
        "id": frame,
        "file_name": f"frame_{frame:05}.jpg"
    })

# ADD annotations
for r in tracking_results:
    coco["annotations"].append({
        "id": ann_id,
        "image_id": r["frame"],
        "category_id": r["category_id"],
        "bbox": r["bbox"],
        "score": r["score"],
        "iscrowd": 0
    })
    ann_id += 1

with open("deepsort_coco.json", "w") as f:
    json.dump(coco, f, indent=4)

print("COCO file saved → deepsort_coco.json")


In [ ]:
from google.colab import files
files.download("deepsort_coco.json")


In [ ]:
from google.colab import files
uploaded = files.upload()
